In [33]:
import pandas as pd
import numpy as np
import datetime
from googleads import adwords
import google_analytics_reporting_api as ga
import os
import logging

In [34]:
today_Friday=datetime.datetime.now().date()
today_weekday_num_py=today_Friday.weekday()
this_week_Monday=today_Friday-datetime.timedelta(days=today_weekday_num_py)

week_counter_str=str(int(np.ceil((this_week_Monday-datetime.date(2019,1,16)).days/7)))
week="_Jan_Week_"+week_counter_str
week

'_Jan_Week_2'

In [42]:
logging.basicConfig(filename='saatva_celery.log', level=logging.INFO)
logging.info("Start running: " +str(datetime.datetime.now()))
today_Friday=datetime.datetime.now().date()
today_weekday_num_py=today_Friday.weekday()
this_week_Monday=today_Friday-datetime.timedelta(days=today_weekday_num_py)

week_counter_str=str(int(np.floor((this_week_Monday-datetime.date(2019,1,16)).days/7+1)))
week="_Jan_Week_"+week_counter_str

Start_Date_2=str(this_week_Monday-datetime.timedelta(days=4)) # To be change
End_Date_2=str(datetime.datetime.strptime(Start_Date_2,"%Y-%m-%d").date()+datetime.timedelta(days=6))

Start_Date_overall='2019-01-01'
End_Data_overall=End_Date_2

Start_Date_1='2019-01-01' # No Change
End_Date_1='2019-01-16'

today_str=str(datetime.datetime.now().date())

In [44]:
Start_Date_2

'2019-01-24'

In [36]:
analytics = ga.initialize_analyticsreporting('/home/jian/Projects/Saatva/Summary_Table/GA_api_jubapluscc@gmail.com.json')
    
def num_func_trans(df):
    if 'ga:sessions' in df.columns:
        df['ga:sessions']=df['ga:sessions'].apply(lambda x: int(x.replace(",","")))
        df['ga:sessions']=df['ga:sessions'].astype(int)

    if 'ga:adClicks' in df.columns:
        df['ga:adClicks']=df['ga:adClicks'].apply(lambda x: int(x.replace(",","")))
        df['ga:adClicks']=df['ga:adClicks'].astype(int)

    if 'ga:impressions' in df.columns:
        df['ga:impressions']=df['ga:impressions'].apply(lambda x: int(x.replace(",","")))
        df['ga:impressions']=df['ga:impressions'].astype(int)

    if 'ga:transactions' in df.columns:
        df['ga:transactions']=df['ga:transactions'].apply(lambda x: int(x.replace(",","")))
        df['ga:transactions']=df['ga:transactions'].astype(int)

    if 'ga:transactionRevenue' in df.columns:
        df['ga:transactionRevenue']=df['ga:transactionRevenue'].apply(lambda x: float(x.replace(",","")))
        df['ga:transactionRevenue']=df['ga:transactionRevenue'].astype(float)


    return df

In [37]:
ga_data_1=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='122377905',start_date=Start_Date_1, end_date=End_Date_1,
                                                ga_metrics=['ga:sessions','ga:adClicks','ga:impressions','ga:transactions','ga:transactionRevenue'],
                                                ga_dimensions=['ga:dateHour','ga:sourceMedium'])
    
ga_data_2=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='122377905',start_date=Start_Date_2, end_date=End_Date_2,
                                            ga_metrics=['ga:sessions','ga:adClicks','ga:impressions','ga:transactions','ga:transactionRevenue'],
                                            ga_dimensions=['ga:dateHour','ga:sourceMedium'])


ga_data_saatva=ga_data_1.append(ga_data_2)
ga_data_saatva['View']="Saatva.com"
ga_data_saatva['View_ID']="122377905"
ga_data_saatva.shape
logging.info("1"+str(datetime.datetime.now()))

# In[10]:

# Saatvamattress.com
# dateHour, sourceMedium      sessions, adClicks, impressions
ga_data_1=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='33605697',start_date=Start_Date_1, end_date=End_Date_1,
                                            ga_metrics=['ga:sessions','ga:adClicks','ga:impressions','ga:transactions','ga:transactionRevenue'],
                                            ga_dimensions=['ga:dateHour','ga:sourceMedium'])

ga_data_2=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='33605697',start_date=Start_Date_2, end_date=End_Date_2,
                                            ga_metrics=['ga:sessions','ga:adClicks','ga:impressions','ga:transactions','ga:transactionRevenue'],
                                            ga_dimensions=['ga:dateHour','ga:sourceMedium'])


ga_data_SM=ga_data_1.append(ga_data_2)
ga_data_SM['View']="Saatvamattress.com"
ga_data_SM['View_ID']="33605697"
ga_data_SM.shape
logging.info("2"+str(datetime.datetime.now()))

# In[11]:

ga_data=ga_data_SM.append(ga_data_saatva)
ga_data=num_func_trans(ga_data)


# In[12]:

ga_data['Source']=ga_data['ga:sourceMedium'].apply(lambda x: x.split(" / ")[0])
ga_data['Medium']=ga_data['ga:sourceMedium'].apply(lambda x: x.split(" / ")[1])

In [38]:
ga_data['Defined_Media']=np.nan
ga_data['Defined_Media']=np.where(((ga_data['ga:sourceMedium'].str.contains('youtube.')) | (ga_data['Medium']=="social")),"Social",
                                  np.where(((ga_data['Source'].isin(['msn','instagram','Instagram','pinterest','facebook'])) & (ga_data['Medium']=='cpc')),"Social",
                                           np.where(ga_data['ga:sourceMedium'].str.contains('criteo'),"Retarget",
                                                    np.where(ga_data['Source']=="(direct)","Direct",
                                                             np.where(((ga_data['Source'].isin(['google','bing','yahoo'])) & (ga_data['Medium']=='cpc')),"SEM",
                                                                      np.where(((ga_data['Source'].isin(['google','bing','yahoo'])) & (ga_data['Medium']=='organic')),"SEO",
                                                                               np.where(ga_data['Source']=="powerInbox","Email",
                                                                                        np.where((ga_data['ga:sourceMedium'].str.contains('email')) & (~ga_data['ga:sourceMedium'].isin(["taboola / townnews-wvgazettemail","query.hemailaccessonline.com / referral"]) | (ga_data['ga:sourceMedium'].str.contains("mail.") )),"Email",
                                                                                                 np.where(ga_data['ga:sourceMedium'].str.contains('taboola'),"Video",
                                                                                                          np.where(ga_data['Medium']=="referral","Referral",
                                                                                                                   np.where(ga_data['Medium'].isin(["partner","partners"]),"partner",
                                                                                                                            np.where(ga_data['ga:sourceMedium'].str.contains('display'),"Display","Others")
                                                                                                                            )
                                                                                                                   )
                                                                                                          )
                                                                                                )
                                                                                       )
                                                                              )
                                                                     )
                                                            )
                                                   )
                                          )
                                 )

In [39]:
ga_agg_session=ga_data.groupby(['ga:dateHour','Defined_Media'])['ga:sessions'].sum().to_frame()
ga_agg_session.reset_index(inplace=True)
ga_agg_wide_session=ga_agg_session.pivot('ga:dateHour','Defined_Media','ga:sessions')
ga_agg_wide_session.reset_index(inplace=True)
ga_agg_wide_session['time_hour']=ga_agg_wide_session['ga:dateHour'].apply(lambda x:datetime.datetime.strptime(x,"%Y%m%d%H"))


In [41]:
media_col=sorted(ga_agg_wide_session.columns.tolist())
media_col

['Direct',
 'Display',
 'Email',
 'Others',
 'Referral',
 'Retarget',
 'SEM',
 'SEO',
 'Social',
 'Video',
 'ga:dateHour',
 'partner',
 'time_hour']